In [23]:
import numpy as np
from scipy.linalg import null_space

In [24]:
def create_bootstrap_matrix(contract_ranges):
    m = len(contract_ranges)
    n = max([t[1] for t in contract_ranges])
    A = np.zeros((m, n))
    for i, contract in enumerate(contract_ranges):
        start_col, end_col = contract
        num_cols = end_col - start_col
        weight = 1.0/num_cols
        for j in range(start_col, end_col):
            A[i, j] = weight
    return A

In [45]:
def calc_bootstrap_example(contracts, flat_price):
    A = create_bootstrap_matrix(contracts)
    print('A')
    print(A)
    prices = np.array((flat_price, flat_price))
    print('prices')
    print(prices)
    pinv = np.linalg.pinv(A)
    print('pseudo-inverse')
    print(pinv)
    bootstrapped_prices = np.matmul(pinv, prices)
    print('bootstrapped')
    print(bootstrapped_prices)
    print('average of bootstrapped prices')
    print(np.mean(bootstrapped_prices))
    print('Length of bootstrapped prices')
    print(np.dot(bootstrapped_prices, bootstrapped_prices))
    print('Length of flat price vector')
    flat_price_array = np.full(bootstrapped_prices.shape, flat_price)
    print(np.dot(flat_price_array, flat_price_array))

In [48]:
# Badly Behaved Example
contracts = [
    (0, 3),
    (2, 4),
]
flat_price = 10
print('Badly Behaved Example')
calc_bootstrap_example(contracts, flat_price)

Badly Behaved Example
A
[[0.33333333 0.33333333 0.33333333 0.        ]
 [0.         0.         0.5        0.5       ]]
prices
[10 10]
pseudo-inverse
[[ 1.2 -0.4]
 [ 1.2 -0.4]
 [ 0.6  0.8]
 [-0.6  1.2]]
bootstrapped
[ 8.  8. 14.  6.]
average of bootstrapped prices
8.999999999999996
Length of bootstrapped prices
359.9999999999997
Length of flat price vector
400


In [47]:
# Well Behaved Example
contracts = [
    (0, 4),
    (2, 4),
]
flat_price = 10.0
print('Badly Behaved Example')
calc_bootstrap_example(contracts, flat_price)

Badly Behaved Example
A
[[0.25 0.25 0.25 0.25]
 [0.   0.   0.5  0.5 ]]
prices
[10. 10.]
pseudo-inverse
[[ 2.00000000e+00 -1.00000000e+00]
 [ 2.00000000e+00 -1.00000000e+00]
 [-2.77555756e-16  1.00000000e+00]
 [-2.77555756e-16  1.00000000e+00]]
bootstrapped
[10. 10. 10. 10.]
average of bootstrapped prices
9.999999999999998
Length of bootstrapped prices
399.9999999999999
Length of flat price vector
400.0


In [58]:
def calc_bootstrap_example_with_nullspace(contracts, flat_price):
    A = create_bootstrap_matrix(contracts)
    print('A')
    print(A)
    prices = np.array((flat_price, flat_price))
    print('prices')
    print(prices)
    pinv = np.linalg.pinv(A)
    print('pseudo-inverse')
    print(pinv)
    ns = null_space(A)
    print('nullspace basis')
    print(ns)
    ns_proj_mat = np.matmul(ns, ns.T)
    print('nullspace project matrix')
    print(ns_proj_mat)
    target = np.array([[flat_price], [flat_price], [flat_price], [flat_price]])
    print('target')
    print(target)
    target_proj_into_ns = np.matmul(ns_proj_mat, target)
    print('target projected into nullspace')
    print(target_proj_into_ns)
    bootstrapped_prices = np.matmul(pinv, prices) + target_proj_into_ns.T
    print('bootstrapped')
    print(bootstrapped_prices)
    print('average of bootstrapped prices')
    print(np.mean(bootstrapped_prices))
    print('Length of bootstrapped prices')
    print(np.dot(bootstrapped_prices, bootstrapped_prices))
    print('Length of flat price vector')
    flat_price_array = np.full(bootstrapped_prices.shape, flat_price)
    print(np.dot(flat_price_array, flat_price_array))

In [59]:
# Badly Behaved Example With Nullspace Component
contracts = [
    (0, 3),
    (2, 4),
]
flat_price = 10
print('Badly Behaved Example With Nullspace Component')
calc_bootstrap_example_with_nullspace(contracts, flat_price)

Badly Behaved Example With Nullspace Component
A
[[0.33333333 0.33333333 0.33333333 0.        ]
 [0.         0.         0.5        0.5       ]]
prices
[10 10]
pseudo-inverse
[[ 1.2 -0.4]
 [ 1.2 -0.4]
 [ 0.6  0.8]
 [-0.6  1.2]]
nullspace basis
[[-0.74489571 -0.21243911]
 [ 0.33825438  0.69683856]
 [ 0.40664134 -0.48439945]
 [-0.40664134  0.48439945]]
nullspace project matrix
[[ 0.6 -0.4 -0.2  0.2]
 [-0.4  0.6 -0.2  0.2]
 [-0.2 -0.2  0.4 -0.4]
 [ 0.2  0.2 -0.4  0.4]]
target
[[10]
 [10]
 [10]
 [10]]
target projected into nullspace
[[ 2.]
 [ 2.]
 [-4.]
 [ 4.]]
bootstrapped
[[10. 10. 10. 10.]]
average of bootstrapped prices
9.999999999999995
Length of bootstrapped prices


ValueError: shapes (1,4) and (1,4) not aligned: 4 (dim 1) != 1 (dim 0)

In [26]:
svd = np.linalg.svd(A)
svd

(array([[ 0.        ,  1.        ,  0.        ],
        [ 0.70710678,  0.        , -0.70710678],
        [ 0.70710678,  0.        ,  0.70710678]]),
 array([0.8660254 , 0.57735027, 0.5       ]),
 array([[ 1.28197512e-16, -1.75121059e-16,  4.69235462e-17,
          4.08248290e-01,  8.16496581e-01,  4.08248290e-01],
        [ 5.77350269e-01,  5.77350269e-01,  5.77350269e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.61481343e-17, -1.31340794e-16,  3.51926597e-17,
         -7.07106781e-01,  2.77555756e-16,  7.07106781e-01],
        [ 1.72546030e-01, -6.43950551e-01,  4.71404521e-01,
          3.33333333e-01, -3.33333333e-01,  3.33333333e-01],
        [ 6.43950551e-01, -4.71404521e-01, -1.72546030e-01,
         -3.33333333e-01,  3.33333333e-01, -3.33333333e-01],
        [ 4.71404521e-01,  1.72546030e-01, -6.43950551e-01,
          3.33333333e-01, -3.33333333e-01,  3.33333333e-01]]))

In [27]:
ns = null_space(A)
ns

array([[ 0.17254603,  0.64395055,  0.47140452],
       [-0.64395055, -0.47140452,  0.17254603],
       [ 0.47140452, -0.17254603, -0.64395055],
       [ 0.33333333, -0.33333333,  0.33333333],
       [-0.33333333,  0.33333333, -0.33333333],
       [ 0.33333333, -0.33333333,  0.33333333]])

In [28]:
target = np.array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0])
print('target')
print(target)
error = target - bootstrapped_prices
print('error')
print(error)


target
[10. 10. 10. 10. 10. 10.]
error
[-1.77635684e-15  3.55271368e-15  0.00000000e+00  3.33333333e+00
 -3.33333333e+00  3.33333333e+00]


In [29]:
c = np.matmul(ns.T, error)
print(c)

[ 3.33333333 -3.33333333  3.33333333]


In [30]:
y = bootstrapped_prices + np.matmul(ns, c)
y

array([10., 10., 10., 10., 10., 10.])

https://github.com/cmdty/curves/issues/8

In [31]:
import datetime
from curves import bootstrap_contracts
contracts = [# Day contracts
 (datetime.date(2020, 4, 3), datetime.date(2020, 4, 3), 18.02),
 (datetime.date(2020, 4, 6), datetime.date(2020, 4, 6), 12.73),
 # Week contracts
 (datetime.date(2020, 4, 6), datetime.date(2020, 4, 12), 16.0),
 (datetime.date(2020, 4, 13), datetime.date(2020, 4, 19), 16.78),
 (datetime.date(2020, 4, 20), datetime.date(2020, 4, 26), 20.82),
 (datetime.date(2020, 4, 27), datetime.date(2020, 5, 3), 17.97), # This is what is causing issues
 # Month contracts
 (datetime.date(2020, 5, 1), datetime.date(2020, 5, 31), 22.21),
 (datetime.date(2020, 6, 1), datetime.date(2020, 6, 30), 27.3)]

piecewise_curve, bootstrapped_contracts = bootstrap_contracts(contracts, freq='D')
print(piecewise_curve['2020-04-25':'2020-05-05'])

2020-04-25    20.820000
2020-04-26    20.820000
2020-04-27    16.094423
2020-04-28    16.094423
2020-04-29    16.094423
2020-04-30    16.094423
2020-05-01    20.470769
2020-05-02    20.470769
2020-05-03    20.470769
2020-05-04    22.396346
2020-05-05    22.396346
Freq: D, dtype: float64


In [32]:
print(bootstrapped_contracts)

[Contract(start=Period('2020-04-03', 'D'), end=Period('2020-04-03', 'D'), price=18.02), Contract(start=Period('2020-04-06', 'D'), end=Period('2020-04-06', 'D'), price=12.730000000000008), Contract(start=Period('2020-04-07', 'D'), end=Period('2020-04-12', 'D'), price=16.545), Contract(start=Period('2020-04-13', 'D'), end=Period('2020-04-19', 'D'), price=16.78), Contract(start=Period('2020-04-20', 'D'), end=Period('2020-04-26', 'D'), price=20.819999999999993), Contract(start=Period('2020-04-27', 'D'), end=Period('2020-04-30', 'D'), price=16.09442307692308), Contract(start=Period('2020-05-01', 'D'), end=Period('2020-05-03', 'D'), price=20.470769230769214), Contract(start=Period('2020-05-04', 'D'), end=Period('2020-05-31', 'D'), price=22.396346153846146), Contract(start=Period('2020-06-01', 'D'), end=Period('2020-06-30', 'D'), price=27.299999999999994)]


---
https://github.com/cmdty/curves/issues/12


In [33]:
test_contracts = [(datetime.date(2020, 8, 24), datetime.date(2020, 8, 30), 10.0), 
(datetime.date(2020, 8, 31), datetime.date(2020, 9, 6), 10.0),
(datetime.date(2020, 9, 1), datetime.date(2020, 9, 30), 10.0)]

test_piecewise_curve, test_bootstrapped_contracts = bootstrap_contracts(test_contracts, freq='D')
print(test_piecewise_curve)

2020-08-24    10.0
2020-08-25    10.0
2020-08-26    10.0
2020-08-27    10.0
2020-08-28    10.0
2020-08-29    10.0
2020-08-30    10.0
2020-08-31    10.0
2020-09-01    10.0
2020-09-02    10.0
2020-09-03    10.0
2020-09-04    10.0
2020-09-05    10.0
2020-09-06    10.0
2020-09-07    10.0
2020-09-08    10.0
2020-09-09    10.0
2020-09-10    10.0
2020-09-11    10.0
2020-09-12    10.0
2020-09-13    10.0
2020-09-14    10.0
2020-09-15    10.0
2020-09-16    10.0
2020-09-17    10.0
2020-09-18    10.0
2020-09-19    10.0
2020-09-20    10.0
2020-09-21    10.0
2020-09-22    10.0
2020-09-23    10.0
2020-09-24    10.0
2020-09-25    10.0
2020-09-26    10.0
2020-09-27    10.0
2020-09-28    10.0
2020-09-29    10.0
2020-09-30    10.0
Freq: D, dtype: float64


In [9]:
import pandas as pd

date_str = '2020-01-01'
freq='H'

period = pd.Period(date_str, freq=freq)
print(period)
ts = pd.Timestamp(date_str, tz='Europe/London')
print(ts)

offset = pd.tseries.frequencies.to_offset(freq)

print(period - offset)
print(ts - offset)

2020-01-01 00:00
2020-01-01 00:00:00+00:00
2019-12-31 23:00
2019-12-31 23:00:00+00:00


In [16]:
from datetime import date, datetime
pd.Timestamp(pd.Timestamp(2023,1, 1))

Timestamp('2023-01-01 00:00:00')